# Ames Housing Dataset Regression

## Packages and Data Importing

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
if 'presentation' in plt.style.available:
    plt.style.use(['bmh','presentation'])
else:
    plt.style.use('bmh')
    
# Additional Imports:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import Imputer, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, Lasso, ElasticNetCV

In [2]:
import os
os.listdir('data')

['kaggl_data.pickle',
 'sample_submission.csv',
 'submission_1.csv',
 'test.csv',
 'test_public.csv',
 'train.csv',
 'train_public.csv']

In [3]:
train_data = pd.read_csv('data/train.csv', index_col = 'Id')
kaggl_data = pd.read_csv('data/test.csv',  index_col = 'Id')

In [4]:
use_public_data = False
if use_public_data:
    # This is to allow me to run the notebook on the train and test CSVs from
    # the main Kaggle page for this dataset
    
    # BEWARE! COLUMN NAMES ARE DIFFERENT IN PUBLIC DATA :(
    train_data = pd.read_csv('data/train_public.csv', index_col = 'Id')
    kaggl_data = pd.read_csv('data/test_public.csv',  index_col = 'Id')

## Train / Test Split 
(Immediately! This is best, but not always done in practice)

In [5]:
print('Training data has {} rows.'.format(train_data.shape[0]))

Training data has 2051 rows.


In [6]:
X = train_data.drop('SalePrice', axis=1)
y = train_data['SalePrice']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

## Manual Feature Engineering

In [8]:
# Create an 'EDA' dataframe we'll use to do some exploring
EDA = X_train.copy()
EDA['SalePrice'] = y_train

In [9]:
# There are 27 neighborhoods. Let's put them into groups of 9:
neighborhood_ranks = EDA.groupby('Neighborhood')['SalePrice'].mean().sort_values().index

low_neigh  = neighborhood_ranks[:9]
mid_neigh  = neighborhood_ranks[9:18]
high_neigh = neighborhood_ranks[18:]

In [10]:
# Making sure these porch columns are numeric:
EDA[[col for col in EDA.columns if 'Porch' in col]].dtypes

Open Porch SF     int64
Enclosed Porch    int64
3Ssn Porch        int64
Screen Porch      int64
dtype: object

In [11]:
def manual_feature_eng(data):
    '''Some basic manual feature engineering based on EDA of X_train'''
    eng_data = data.copy()
    # Years info:
    eng_data['Years_Old'] = 2018 - eng_data['Year Built']
    eng_data['Garage Age'] = 2018 - eng_data['Garage Yr Blt']
    eng_data['Years Since Sale'] = 2018 - eng_data['Yr Sold']
    eng_data['Years Since Remodel'] = 2018 - eng_data['Year Remod/Add']
    eng_data.drop(['Year Built','Garage Yr Blt','Yr Sold','Year Remod/Add'],
                 axis=1, inplace=True)
    # Neighborhood info:
    eng_data['High_Neigh'] = [1 if x in high_neigh else 0 for x in eng_data['Neighborhood']]
    eng_data['Mid_Neigh'] = [1 if x in mid_neigh else 0 for x in eng_data['Neighborhood']]
    eng_data['Low_Neigh'] = [1 if x in low_neigh else 0 for x in eng_data['Neighborhood']]
    eng_data.drop('Neighborhood', axis=1, inplace=True)
    
    # Is there miscellaneous furniture?
    eng_data['MiscFurn'] = eng_data['Misc Val'] > 0
    return eng_data

In [12]:
X_train = manual_feature_eng(X_train)
X_test = manual_feature_eng(X_test)
kaggl_data = manual_feature_eng(kaggl_data)

## Data Preprocessing: Categorical Data

In [13]:
# Before we begin, let's check to see if there are any columns in the Kaggle 
# set that aren't in the training set:

assert [col for col in kaggl_data.columns if col not in X_train.columns] == []

In [14]:
# And vice versa:

assert [col for col in X_train.columns if col not in kaggl_data.columns] == []

In [15]:
# Which columns have missing values?
# Simple helper function we can run and re-run as we clean:
def show_null_cols():
    missing_vals = X_train.isnull().sum().sort_values(ascending=False)
    return missing_vals[missing_vals > 0]

show_null_cols().head()

Pool QC         1633
Misc Feature    1587
Alley           1527
Fence           1311
Fireplace Qu     791
dtype: int64

In [16]:
X_train['Fence'].value_counts(dropna=False)

NaN      1311
MnPrv     191
GdWo       65
GdPrv      64
MnWw        9
Name: Fence, dtype: int64

In [17]:
# All of our preprocessing will ultimately go here:
def preprocessing(data):
    try:
        cleaned_data = data.drop('PID', axis=1)
    except:
        cleaned_data = data
    fillna_dict = {
        'Pool QC':'No Pool',
        'Alley':'No Alley',
        # Let's let the get_dummies drop 'Misc Features' if NA
        'Fence':'No Fence',
        'Fireplace Qu':'No Fireplace',
        # Lot frontage can be mean imputed
        'Garaga Finish': 'No Garage',
        'Garage Qual': 'No Garage',
        'Garage Cond': 'No Garage',
        'Garage Type': 'No Garage',
        'Bsmt Exposure':'No Garage',
        'BsmtFin Type 2':'No Basement',
        'BsmtFin Type 2':'No Basement',
        'BsmtFin Type 1':'No Basement',
        'Bsmt Cond':'No Basement',
        'Bsmt Qual':'No Basement',
        'Mas Vnr Type':'No Mas Vnr'        
    }
    
    cleaned_data = cleaned_data.fillna(fillna_dict)
    
    return(cleaned_data)
    
X_train = preprocessing(X_train)
X_test  = preprocessing(X_test)
kaggl_data = preprocessing(kaggl_data)

In [18]:
# Grab the string columns:
string_cols = X_train.select_dtypes(exclude=[np.number]).columns

In [19]:
# Get some dummies:
X_train = pd.get_dummies(X_train, columns=string_cols)
X_test = pd.get_dummies(X_test, columns=string_cols)
kaggl_data = pd.get_dummies(kaggl_data, columns=string_cols)

## Addressing Column Mismatch After Dummifying

In [20]:
# Add columns of zeros to test and kaggle sets for columns that *do* appear in
# the training set.

model_cols = X_train.columns

def add_model_cols(data, model_cols):
    new_data = data.copy()
    for missing_col in [col for col in model_cols if col not in data.columns]:
        new_data[missing_col] = 0
    return new_data

X_test = add_model_cols(X_test, model_cols=model_cols)
kaggl_data = add_model_cols(kaggl_data, model_cols=model_cols)

In [21]:
# Now, let's only consider columns in X_test and kaggl_data that appear in
# the training set. We'll call these 'model columns':

kaggl_data = kaggl_data[model_cols]
X_test     = X_test[model_cols]

In [22]:
# Make sure we've done this correctly:
assert X_train.shape[1] == X_test.shape[1] == kaggl_data.shape[1]
assert X_train.columns.all() == X_test.columns.all()== kaggl_data.columns.all() 

## Imputing Numerical Missing Data: Handling Numerical Data

In [23]:
imp = Imputer(strategy='mean')
imp.fit(X_train)
X_train = imp.transform(X_train)
X_test  = imp.transform(X_test)
kaggl_data = imp.transform(kaggl_data)

In [24]:
def array_null_check(array):
    '''Turns an array into a dataframe so that we can check for null values'''
    return pd.DataFrame(array).isnull().sum().sum()

In [25]:
assert array_null_check(X_train) == array_null_check(X_test) \
                                 == array_null_check(kaggl_data)

## Brute Force Feature Engineering

In [26]:
brute = True
if brute:
    pf = PolynomialFeatures(interaction_only=True)
    X_train = pf.fit_transform(X_train)
    X_test  = pf.transform(X_test)
    kaggl_data = pf.transform(kaggl_data)

In [27]:
# Maybe this is too many columns???
X_train.shape

(1640, 38227)

## Scaling

In [28]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test  = ss.transform(X_test)
kaggl_data = ss.transform(kaggl_data)

In [29]:
print(np.apply_along_axis(np.mean, axis=1, arr=X_train).mean())
print(np.apply_along_axis(np.mean, axis=1, arr=X_test).mean())
print(np.apply_along_axis(np.mean, axis=1, arr=kaggl_data).mean())

-7.51431437399e-18
0.00521022034256
0.0101429929682


In [30]:
print(np.apply_along_axis(np.std, axis=1, arr=X_train).mean())
print(np.apply_along_axis(np.std, axis=1, arr=X_test).mean())
print(np.apply_along_axis(np.std, axis=1, arr=kaggl_data).mean())

0.715895738208
0.945378726735
2.17472892682


## Feature Elimination

In [31]:
from sklearn.feature_selection import VarianceThreshold, SelectPercentile, f_regression

In [32]:
if brute:
    # Only do feature elimination if feature engineering happened by brute force
    feature_variances = np.apply_along_axis(np.var, axis=0, arr= X_train)

    # Define a percentile threshold. Do I want the top 1% of features by variance?
    perc_thresh = np.percentile(feature_variances, 99)
    perc_thresh

    vt = VarianceThreshold(threshold=perc_thresh)
    X_train_reduced = vt.fit_transform(X_train)
    X_test_reduced  = vt.transform(X_test)
    kaggl_reduced   = vt.transform(kaggl_data)
    print(X_train_reduced.shape[1])
else:
    X_train_reduced = X_train
    X_test_reduced  = X_test
    kaggl_reduced   = kaggl_data

1616


In [33]:
# Or do I want to select the top 1% of features according 
# to the f_regression function?

# sp = SelectPercentile(score_func=f_regression, percentile = 1)
# X_train_reduced = sp.fit_transform(X_train, y_train)
# X_test_reduced  = sp.transform(X_test)
# kaggl_reduced   = sp.transform(kaggl_data)
# print(X_train.shape[1])

## Modeling

### Linear Regression

In [34]:
run = True
if run:
    lin = LinearRegression()
    lin.fit(X_train_reduced, y_train)
    cv_scores = cross_val_score(lin, X_train_reduced, y_train, cv=3).mean()

    print('{} model has average performance of {}'
          .format(str(lin).split('(')[0], cv_scores.mean()))

LinearRegression model has average performance of -1.1090739859232431e+23


### Ridge Regression

In [35]:
run = False
if run:
    rid = RidgeCV()
    rid.fit(X_train_reduced, y_train)
    cv_scores = cross_val_score(rid, X_train_reduced, y_train, cv=3).mean()

    print('{} model has average performance of {}'
          .format(str(rid).split('(')[0], cv_scores.mean()))

### Lasso Regression

In [36]:
run = True
if run:
    # Define a reasonable range of alphas based on previous LASSO fits:
    alphas = np.logspace(2,4,20)
    las = LassoCV(alphas=alphas, n_jobs=-1)
    las.fit(X_train_reduced, y_train)
    cv_scores = cross_val_score(las, X_train_reduced, y_train, cv=3).mean()
    best_alpha = las.alpha_
    print('{} model has average performance of {}'
          .format(str(las).split('(')[0], cv_scores.mean()))

C:\Users\benps\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\benps\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\benps\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


LassoCV model has average performance of 0.7841585774774625


In [37]:
las = Lasso(alpha=best_alpha, max_iter=2000)
cv_scores = cross_val_score(las, X_train_reduced, y_train, cv=3).mean()
las.fit(X_train_reduced, y_train)
print('{} model has average performance of {}'
      .format(str(las).split('(')[0], cv_scores.mean()))

Lasso model has average performance of 0.7955884976778741


### ElasticNet Regression

In [38]:
run = False
if run:
    ela = ElasticNetCV(n_alphas=10)
    ela.fit(X_train_reduced, y_train)
    cv_scores = cross_val_score(ela, X_train_reduced, y_train, cv=3).mean()

    print('{} model has average performance of {}'
          .format(str(ela).split('(')[0], cv_scores.mean()))

## Final Model Test

In [39]:
try:
    print('Test set performance of {}: {}'.format(str(lin).split('(')[0],lin.score(X_test_reduced, y_test)))
except:
    pass    

try:
    print('Test set performance of {}: {}'.format(str(rid).split('(')[0],rid.score(X_test_reduced, y_test)))
except:
    pass    

try:
    print('Test set performance of {}: {}'.format(str(las).split('(')[0],las.score(X_test_reduced, y_test)))
except:
    pass          

try:
    print('Test set performance of {}: {}'.format(str(ela).split('(')[0],ela.score(X_test_reduced, y_test)))
except:
    pass   

Test set performance of LinearRegression: -3.375603848304643e+24
Test set performance of Lasso: 0.8637319667132236


## Choosing a Model and Making Submission:

In [40]:
# Choose a model based on test set performance:
chosen_model = las 

In [41]:
kaggl_preds = chosen_model.predict(kaggl_reduced)

In [42]:
kaggl_id = pd.read_csv('data/test.csv')['Id']

In [43]:
sample_submission = pd.read_csv('data/sample_submission.csv')
submission_columns= sample_submission.columns
sample_submission.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [44]:
submission = pd.DataFrame({submission_columns[0]:kaggl_id,
                           submission_columns[1]:kaggl_preds})
submission.head()

,Id,SalePrice
0,2658,85507.239989
1,2718,150141.634905
2,2414,214020.889623
3,1989,86717.542569
4,625,187801.029727


In [45]:
# submission.to_csv('data/submission_1.csv', index=False) 